<a href="https://colab.research.google.com/github/BOSOEK/Machine_Learning_with_Book/blob/main/Deep_Learning_Chatbot_for_First_Time/Models/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개채명 인식(Named Entitiy Recognition) - NER
> __문장 내의 단어가 인물, 장소, 날짜 등을 의미__하는지 인식하는 것으로 이런 프로그램을 __개채명 인식기__ 라고 한다

### BIO 표기법
> 각 토큰마다 태그를 붙이기 위해 사용
* B(Beggining) : 개체명 __시작 단어__에 'B-개체명'으로 태그된다.
* I(Inside) : B-개체명과 __연결되는 단어__일때 'I-개체명'으로 태그된다.
* O(Outside) : __개체명 이외__의 모든 토큰에 태그된다

In [9]:
# Bi-LSTM 모델 생성, 개체명 예측 예제
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

# 학습 데이터 불러오는 함수
def read_file(file_name):
    # 라인별로 토큰 번호, 단어 토큰, 품사 태그, BIO 태그 정보를 불러온다.
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, I in enumerate(lines):
            if I[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif I[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif I[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(I.split()))
    return sents

# 학습용 데이터 불러오기
corpus = read_file('train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)   # 원본 문장의 분리된 단어 토큰 저장
    tags.append(bio_tag)            # Bio 태그 정보들이 저장
print('샘플 크기 : \n', len(sentences))
print('0번째 샘플 문장 시퀀스 : \n', sentences[0])
print('0번째 샘플 bio 태그 : \n', tags[0])
print('샘플 문장 시퀀스 최대 길이 : ', max(len(l) for l in sentences))
print('샘플 문장 시퀀스 평균 길이 : ', (sum(map(len, sentences)) / len(sentences)))

# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')  # 첫번째 인덱스에 OOV 사용
sent_tokenizer.fit_on_texts(sentences)    # 앞에서 만들어진 단어 시퀀스와 태그 시퀀스를 사전으로 만들기 위해 토크나이저를 정의후 fit_on_texts 호출
tag_tokenizer = preprocessing.text.Tokenizer(lower=False)  # 태그 정보는 소문자로 변환 X
tag_tokenizer.fit_on_texts(tags)

# 단어 사전 & 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print('BIO 태그 사전 크기 : ', tag_size)
print('단어 사전 크기 : ', vocab_size)

# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

# index to word / NER 정의
index_to_word = sent_tokenizer.index_word   # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word       # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 학습 & 테스트 데이터 8:2로
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)
print('학습 샘플 시퀀스 형상 : ', x_train.shape)
print('학습 샘플 레이블 형상 : ', y_train.shape)
print('테스트 샘플 시퀀스 형상 : ', x_test.shape)
print('테스트 샘플 레이블 형상 : ', y_test.shape)

# 모델 제작 : Bi-LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)
print('모델 평가 : ', model.evaluate(x_test, y_test)[1])
# 모델 평가시에 O 태그의 남용으로 성능과 무관하게 정확도가 높아지기에
# 개체명 인식에 사용되는 성능평가인 F1 스코어를 사용해야한다.

# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace('PAD', '0'))
        result.append(temp)
    return result

# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)                # (711, 40) => model => (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted)      # 예측된 NER
test_tags = sequences_to_tag(y_test)

# F1 스코어 계산
from seqeval.metrics import f1_score, classification_report
# classification_report 함수로 NER 태그별 계산 정밀도 & 재현율 & F1 스코어 출력
print(classification_report(test_tags, pred_tags))
print('F1-score : {:.1%}'.format(f1_score(test_tags, pred_tags)))

# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어 : OOV
        new_x.append(word_to_index['OOV'])

print('새로운 시퀀스 : ', new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0, maxlen=max_len)

# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1)  # 예측된 NER 인덱스값 추출

print('{:10} {:5}'.format('단어', '예측된 NER'))
print('-' * 50)
for w, pred in zip(new_sentence, p[0]):
    print('{:10} {:5}'.format(w, index_to_ner[pred]))

샘플 크기 : 
 1995
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 :  143
샘플 문장 시퀀스 평균 길이 :  30.727318295739348
BIO 태그 사전 크기 :  8
단어 사전 크기 :  9004
[183, 12, 2552, 436, 100, 808, 348, 10, 1911, 7, 918, 551, 104, 719, 372, 4, 3948, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
학습 샘플 시퀀스 형상 :  (1596, 40)
학습 샘플 레이블 형상 :  (1596, 40, 8)
테스트 샘플 시퀀스 형상 :  (399, 40)
테스트 샘플 레이블 형상 :  (399, 40, 8)
Epoch 1/10
13/13 [==============================] - 17s 838ms/step - loss: 0.5358 - accuracy: 0.8149
Epoch 2/10
13/13 [==============================] - 11s 830ms/step - loss: 0.3072 - accuracy: 0.8753
Epoch 3/10
13/13 [==============================] - 11s 828ms/step - loss: 0.1939 - accuracy: 0.9041
Epoch 4/10
13/13 [==============================] - 11s 830ms/s

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWar

              precision    recall  f1-score   support

           _       0.52      0.46      0.49       259
         _DT       0.77      0.87      0.81       144
         _LC       0.74      0.42      0.54       170
         _OG       0.76      0.56      0.65       252
         _PS       0.78      0.42      0.54       209
         _TI       0.73      0.70      0.72        27

   micro avg       0.69      0.53      0.60      1061
   macro avg       0.72      0.57      0.62      1061
weighted avg       0.70      0.53      0.59      1061

F1-score : 60.1%
새로운 시퀀스 :  [2379, 1195, 3546, 239, 8211, 4453, 1]
단어         예측된 NER
--------------------------------------------------
삼성전자       B_OG 
출시         O    
스마트폰       O    
오늘         B_DT 
애플         I    
도전장        I    
내밀다.       I    


### F1 스코어 계산 요소
* 정확도 : 실제 정답과의 유사도
* 정밀도 : 결과값이 얼마나 일정하게 분포하는가
* 재현율 : 정답인 것들중 모델이 정답이라 예측한 비율

>F1 스코어는 정밀도와 재현율의 조화 평균이다

__F1 score = 2 * ((정밀도 * 재현율) / (정밀도 + 재현율))__